In [1]:
import numpy as np
import os
from glob import glob

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import TensorBoard

import imgaug
from sklearn.model_selection import train_test_split

from utils_dh import utils
from python.common.NnetsX import NNets ,MyReLU

import cv2
import math

import datetime
import pickle as pk

import matplotlib.pyplot as plt

2.6.0
tensorflow
channels_last


In [2]:
gpu = 0
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.set_visible_devices(gpus[gpu], 'GPU')
        tf.config.experimental.set_memory_growth(gpus[gpu], True)
    except RuntimeError as e:
        print(e)

tf.keras.backend.set_image_data_format('channels_last')

BATCH_NORMALIZATION_NO = -1
BATCH_NORMALIZATION_YES = 0

DOWNSAMPLING_MAXPOOL = 0
DOWNSAMPLING_STRIDED_CONV = 1

UPSAMPLING_UPSAMPLE = 0
UPSAMPLING_TRANSPOSED_CONV = 1

batch_size = 8
epochs = 300
n_channel = 4
input_shape = (256,256,n_channel)

model_dir = '/docker/FluSegTip_1104/models/1104_111522/model_best.h5'


nnets = NNets()
nnets.m_SamePartActivation = MyReLU
nnets.m_RegularizerL1L2 = False
nnets.m_Dropout = 0.5
nnets.m_Residual = True
nnets.m_BatchNormalization = BATCH_NORMALIZATION_YES
nnets.m_BorderMode = "same"
nnets.m_Initialization = "glorot_uniform"
nnets.m_DownSampling = DOWNSAMPLING_STRIDED_CONV
nnets.m_UpSampling = UPSAMPLING_UPSAMPLE
nbUsedChannel = n_channel
nbStartFilter = 8
kernelSize = 3
nbConvPerLayer = [2, 2, 2, 2, 2, 2, 2]
nbDeconvPerLayer = [2, 2, 2, 2, 2, 2]

optimizer = SGD(lr=0.01, decay=5e-4, momentum=0.99)

model = nnets.DefineDeepUVNet(input_shape, _nbFilters=nbStartFilter, _kernelSize=kernelSize,
                              _convPerLevel=nbConvPerLayer, _upConvPerLevel=nbDeconvPerLayer, _optimizer=optimizer)

model.load_weights(model_dir)

2021-11-05 06:24:55.003190: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-05 06:24:55.010916: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-05 06:24:55.012606: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
/usr/local/lib/python3.8/dist-packages/keras/optimizer_v2/optimizer_v2.py:355: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(
2021-11-05 06:24:55.023337: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to 

In [3]:
scale_add = 0.1
trans = 0.16
rot = 9.0
cval = 0.0
fprob = 0.5

scale = {"x": (1.-scale_add, 1.+scale_add), "y": (1.-scale_add, 1.+scale_add)}
translation = {"x": (-trans, trans), "y": (-trans, trans)}
rotate = (-rot, rot) # Degree

ps_dict_root = '/docker/i2i/EntireData/entire_list_211026.pkl'
ps_dict = pk.load(open(ps_dict_root, 'rb'))
ps_root = '/docker/i2i/EntireData'

hyper = {'Width':256, 'Height':256,
         'FlipProb':fprob,
         'AffineScale':scale,
         'AffineTrans':translation,
         'AffineRot':rotate,
         'AffineCval': cval,
         'Dataset':[ps_dict_root, ps_root],
         'ModelSavingPath': model_dir,
         'BatchSize':batch_size,
         'Epochs':epochs}

In [4]:
test_gen = utils.entireDataGen(ps_dict['test'], ps_root, hyper, batch_size=batch_size, valid=True, tipOnly=True)
res = model.evaluate(test_gen, workers=8)

2021-11-05 00:39:28.972875: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2021-11-05 00:39:30.470386: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8202


623/623 [==============================] - 12s 14ms/step - loss: -0.7227 - DiceCoef: 0.7227


In [5]:
res

[-0.7227000594139099, 0.722727358341217]

In [6]:
ps_dict['test']

{3: [3],
 12: [1],
 36: [1],
 63: [1],
 43: [1],
 11: [2, 1],
 20: [1],
 51: [1],
 30: [2],
 54: [1],
 47: [1],
 15: [1],
 60: [1],
 48: [1],
 35: [1],
 22: [1],
 55: [1],
 56: [1],
 101: [1],
 44: [1],
 1: [1],
 57: [1],
 40: [1],
 2: [1],
 17: [1],
 42: [1],
 49: [1],
 5: [1],
 52: [1],
 61: [2],
 7: [1],
 45: [1],
 6: [1]}

In [7]:
#del test_gen
test_ps_gen = utils.entireDataGen(ps_dict['test'], ps_root, hyper, batch_size=batch_size, valid=True, tipOnly=True, target_p=3, target_s=3, shuffle=False)

In [13]:
def save_test(p, s, model, ps_dict, folder='fluoroSeg'):
    gen = utils.entireDataGen(ps_dict, ps_root, hyper, batch_size=batch_size, valid=True, tipOnly=True, target_p=p, target_s=s, shuffle=False)
    imgs = model.predict(gen, workers=8)
    
    for i in range(imgs.shape[0]):
        img = imgs[i,...,0] * 255
        img = img.astype(np.uint8)
        img = cv2.bitwise_not(img)
        path = gen.x_lists[i][0]
        new_path = path.replace('fluoroReal', folder)
        save_folder = os.path.split(new_path)[0]
        if not os.path.exists(save_folder):
            os.makedirs(save_folder)
        #print(new_path)
        
        cv2.imwrite(new_path, img)


In [14]:
for p in ps_dict['test'].keys():
    for s in ps_dict['test'][p]:
        print(p, s)
        save_test(p, s, model, ps_dict['test'])
        

3 3
12 1
36 1
63 1
43 1
11 2
11 1
20 1
51 1
30 2
54 1
47 1
15 1
60 1
48 1
35 1
22 1
55 1
56 1
101 1
44 1
1 1
57 1
40 1
2 1
17 1
42 1
49 1
5 1
52 1
61 2
7 1
45 1
6 1


In [15]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 4) 0                                            
__________________________________________________________________________________________________
gaussian_noise (GaussianNoise)  (None, 256, 256, 4)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 256, 256, 8)  40          gaussian_noise[0][0]             
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 256, 256, 8)  584         conv2d[0][0]                     
______________________________________________________________________________________________

In [4]:
from IPython.display import SVG
from tensorflow.keras.utils import model_to_dot

SVG(model_to_dot(model).create(prog='dot', format='svg'))

('You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) ', 'for plot_model/model_to_dot to work.')


AttributeError: 'NoneType' object has no attribute 'create'